In [ ]:
# python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7gripper'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

DAT_ROOT = "stowing-deep"
RES_ROOT = "stowing-deep-result"

TEST_DATA_PATH = os.path.join(DATA_PATH, DAT_ROOT)
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, RES_ROOT)
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

def run_test_loop(FILE_OPTION, I_START, I_END, common_args):
    N_DAT = I_END-I_START
    ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
    ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
    RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
    file_list = sorted(os.listdir(DATASET_PATH))

    gtimer = GlobalTimer.instance()
    gtimer.reset(scale=1, timeunit='s')

    gtimer.tic("full_loop")
    for fid in range(I_START, I_END):
        e_step_list = []
        for i_c, cname in enumerate(CNAME_LIST):
            gtimer.tic("step")
            p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                                  '--data_idx', str(fid), '--file_option', FILE_OPTION, '--cname', cname,]
                                 + common_args)
            p.wait()
            e_step = np.round(gtimer.toc("step"), 1)
            e_step_list.append(e_step)
            print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")

        print("")
        print("========== {} / {} (eta: {} / {} s) =========".format(
            fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

## Common params

In [ ]:
RTYPE = "indy7gripper"
DAT_DIR = '20210907-173519'
I_START = 0
I_END = 20


CNAME_LIST = ['None', 'Tool', 'ToolReach', 'Full'] #,'Pairwise']
# CNAME_LIST = ['Full'] #,'Pairwise']
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100

GRASP_SAMPLE = 30
STABLE_SAMPLE = 50
SEARCH_SAMPLE_RATIO= 100
MAX_SKELETONS = 30
TIMED_COMPLETE = True
STACK_TIMELOG = True

common_args = [
            '--rtype', RTYPE,
            '--dat_root', DAT_ROOT,
            '--res_root', RES_ROOT,
            '--dat_dir', DAT_DIR,
            '--VISUALIZE', str(VISUALIZE),
            '--PLAY_RESULT', str(PLAY_RESULT),
            '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
            '--MAX_TIME', str(MAX_TIME),
            '--MAX_ITER', str(MAX_ITER), 
            '--GRASP_SAMPLE', str(GRASP_SAMPLE),
            '--STABLE_SAMPLE', str(STABLE_SAMPLE),
            '--SEARCH_SAMPLE_RATIO', str(SEARCH_SAMPLE_RATIO),
            '--MAX_SKELETONS', str(MAX_SKELETONS),
            '--TIMED_COMPLETE', str(TIMED_COMPLETE),
            '--STACK_TIMELOG', str(STACK_TIMELOG)]

# obj1

In [ ]:
FILE_OPTION = "obj_1"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj1 obs5

In [ ]:
FILE_OPTION = "obj_1_obs3
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj1 obs5

In [ ]:
FILE_OPTION = "obj_1_obs5"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj3

In [ ]:
FILE_OPTION = "obj_3"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj3_pole

In [ ]:
FILE_OPTION = "obj_3_pole"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj3_hard

In [ ]:
FILE_OPTION = "obj_3_hard"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj1 obs0

In [3]:
FILE_OPTION = "obj_1c_obs0"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

KeyboardInterrupt: 

# obj1 obs3

In [ ]:
FILE_OPTION = "obj_1c_obs3"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj1 obs5

In [ ]:
FILE_OPTION = "obj_1c_obs5"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj1 obs7

In [ ]:
FILE_OPTION = "obj_1c_obs7"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj3 obs3

In [ ]:
FILE_OPTION = "obj_3c_obs3"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj3 obs5

In [ ]:
FILE_OPTION = "obj_3c_obs5"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)

# obj3 obs7

In [ ]:
FILE_OPTION = "obj_3c_obs7"
run_test_loop(FILE_OPTION, I_START, I_END, common_args)